# Open3D

In [ ]:
import numpy
import open3d
import open3d.web_visualizer

# from open3d import JVisualizer

points = (numpy.random.rand(1000, 3) - 0.5) / 4
colors = numpy.random.rand(1000, 3)

pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(points)
pcd.colors = open3d.utility.Vector3dVector(colors)

# visualizer = open3d.jvisualizer.JVisualizer()
# visualizer.add_geometry(pcd)
# visualizer.show()
open3d.visualization.draw_geometries([pcd])
# open3d.web_visualizer.draw([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


# Ply file format

In [2]:
import open3d
from pytorch3d.io import load_ply

mesh_file = "cube.ply"

print("Visualizing mesh using Open3D")
mesh = open3d.io.read_triangle_mesh(mesh_file)
open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

print("Loading mesh using PyTorch3D")
vertices, faces = load_ply(mesh_file)
print(f"Type of vertices: {type(vertices)}, Type of faces: {type(faces)}")
print(f"Vertices: {vertices}")
print(f"Faces: {faces}")

Visualizing mesh using Open3D
Loading mesh using PyTorch3D
Type of vertices: <class 'torch.Tensor'>, Type of faces: <class 'torch.Tensor'>
Vertices: tensor([[-1., -1., -1.],
        [ 1., -1., -1.],
        [ 1.,  1., -1.],
        [-1.,  1., -1.],
        [-1., -1.,  1.],
        [ 1., -1.,  1.],
        [ 1.,  1.,  1.],
        [-1.,  1.,  1.]])
Faces: tensor([[0, 1, 2],
        [5, 4, 7],
        [6, 2, 1],
        [3, 7, 4],
        [7, 3, 2],
        [5, 1, 0],
        [0, 2, 3],
        [5, 7, 6],
        [6, 1, 5],
        [3, 4, 0],
        [7, 2, 6],
        [5, 0, 4]])


In [3]:
import open3d
from pytorch3d.io import load_ply

mesh_file = "parallel_plane_mono.ply"

print("Visualizing mesh using Open3D")
mesh = open3d.io.read_triangle_mesh(mesh_file)
open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

print("Loading mesh using PyTorch3D")
vertices, faces = load_ply(mesh_file)
print(f"Type of vertices: {type(vertices)}, Type of faces: {type(faces)}")
print(f"Vertices: {vertices}")
print(f"Faces: {faces}")

Visualizing mesh using Open3D
Loading mesh using PyTorch3D
Type of vertices: <class 'torch.Tensor'>, Type of faces: <class 'torch.Tensor'>
Vertices: tensor([[-1., -1., -1.],
        [ 1., -1., -1.],
        [ 1.,  1., -1.],
        [-1.,  1., -1.],
        [-1., -1.,  1.],
        [ 1., -1.,  1.],
        [ 1.,  1.,  1.],
        [-1.,  1.,  1.]])
Faces: tensor([[0, 1, 2],
        [0, 2, 3],
        [5, 4, 7],
        [5, 7, 6]])


In [4]:
import open3d
from pytorch3d.io import load_ply

mesh_file = "parallel_plane_color.ply"

print("Visualizing mesh using Open3D")
mesh = open3d.io.read_triangle_mesh(mesh_file)
open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

print("Loading mesh using PyTorch3D")
vertices, faces = load_ply(mesh_file)
print(f"Type of vertices: {type(vertices)}, Type of faces: {type(faces)}")
print(f"Vertices: {vertices}")
print(f"Faces: {faces}")

Visualizing mesh using Open3D
Loading mesh using PyTorch3D
Type of vertices: <class 'torch.Tensor'>, Type of faces: <class 'torch.Tensor'>
Vertices: tensor([[-1., -1., -1.],
        [ 1., -1., -1.],
        [ 1.,  1., -1.],
        [-1.,  1., -1.],
        [-1., -1.,  1.],
        [ 1., -1.,  1.],
        [ 1.,  1.,  1.],
        [-1.,  1.,  1.]])
Faces: tensor([[0, 1, 2],
        [0, 2, 3],
        [5, 4, 7],
        [5, 7, 6]])


In [6]:
import open3d
from pytorch3d.io import load_ply

mesh_file = "cube_color.ply"

print("Visualizing mesh using Open3D")
mesh = open3d.io.read_triangle_mesh(mesh_file)
open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

print("Loading mesh using PyTorch3D")
vertices, faces = load_ply(mesh_file)
print(f"Type of vertices: {type(vertices)}, Type of faces: {type(faces)}")
print(f"Vertices: {vertices}")
print(f"Faces: {faces}")

Visualizing mesh using Open3D
Loading mesh using PyTorch3D
Type of vertices: <class 'torch.Tensor'>, Type of faces: <class 'torch.Tensor'>
Vertices: tensor([[-1., -1., -1.],
        [ 1., -1., -1.],
        [ 1.,  1., -1.],
        [-1.,  1., -1.],
        [-1., -1.,  1.],
        [ 1., -1.,  1.],
        [ 1.,  1.,  1.],
        [-1.,  1.,  1.]])
Faces: tensor([[0, 1, 2],
        [5, 4, 7],
        [6, 2, 1],
        [3, 7, 4],
        [7, 3, 2],
        [5, 1, 0],
        [0, 2, 3],
        [5, 7, 6],
        [6, 1, 5],
        [3, 4, 0],
        [7, 2, 6],
        [5, 0, 4]])


In [5]:
import open3d
from pytorch3d.io import load_ply

mesh_file = "cow.ply"

print("Visualizing mesh using Open3D")
mesh = open3d.io.read_triangle_mesh(mesh_file)
open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

print("Loading mesh using PyTorch3D")
vertices, faces = load_ply(mesh_file)
print(f"Type of vertices: {type(vertices)}, Type of faces: {type(faces)}")
print(f"Vertices: {vertices}")
print(f"Faces: {faces}")

Visualizing mesh using Open3D
Loading mesh using PyTorch3D
Type of vertices: <class 'torch.Tensor'>, Type of faces: <class 'torch.Tensor'>
Vertices: tensor([[ 0.6055,  0.1831, -0.4723],
        [ 0.6492,  0.1297, -0.4949],
        [ 0.6011,  0.1055, -0.5333],
        ...,
        [-1.4558,  0.6748,  0.2449],
        [-1.2448,  0.6488,  0.2009],
        [-1.4893,  0.6437,  0.2272]])
Faces: tensor([[   0,    1,    2],
        [   1,    3,    4],
        [   5,    6,    2],
        ...,
        [2370, 2839, 2837],
        [2491, 2013, 2011],
        [2498, 2381, 2433]])


# Wavefront OBJ file format

In [10]:
import open3d
from pytorch3d.io import load_obj

mesh_file = "cube.obj"

print("Visualizing mesh using Open3D")
mesh = open3d.io.read_triangle_mesh(mesh_file)
open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

print("Loading the same file with PyTorch3D")
vertices, faces, aux = load_obj(mesh_file)
print(f"Vertices type: {type(vertices)}")
print(f"Vertices: {vertices}")
print(f"Faces type: {type(faces)}")
print(f"Faces: {faces}")
print(f"Aux type: {type(aux)}")
print(f"Aux: {aux}")

Visualizing mesh using Open3D
Loading the same file with PyTorch3D
Vertices type: <class 'torch.Tensor'>
Vertices: tensor([[-50., -50.,  20.],
        [-50., -50.,  10.],
        [-50.,  50.,  10.],
        [-50.,  50.,  20.],
        [ 50., -50.,  20.],
        [ 50., -50.,  10.],
        [ 50.,  50.,  10.],
        [ 50.,  50.,  20.]])
Faces type: <class 'pytorch3d.io.obj_io.Faces'>
Faces: Faces(verts_idx=tensor([[0, 1, 2],
        [5, 4, 7],
        [6, 2, 1],
        [3, 7, 4],
        [7, 3, 2],
        [5, 1, 0],
        [0, 2, 3],
        [5, 7, 6],
        [6, 1, 5],
        [3, 4, 0],
        [7, 2, 6],
        [5, 0, 4]]), normals_idx=tensor([[-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1]]), textures_idx=tensor([[-1, -1, -1],
        [-1, -1, -1],
        [-1, -1, -1],
  

# Camera

In [ ]:
import open3d
import torch

from pytorch3d.io import load_obj
from pytorch3d.renderer.cameras import PerspectiveCameras, OrthographicCameras

from scipy.spatial.transform import Rotation as Rotation

# Load meshes and visualize it with Open3D
mesh_file = "cube.obj"
print("visualizing the mesh using open3D")
mesh = open3d.io.read_triangle_mesh(mesh_file)
# open3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

# Define a mini-batch of 8 cameras
image_size = torch.ones(8, 2)
image_size[:, 0] = image_size[:, 0] * 1024
image_size[:, 1] = image_size[:, 1] * 512
image_size = image_size.cuda()

focal_length = torch.ones(8, 2)
focal_length[:, 0] = focal_length[:, 0] * 1200
focal_length[:, 1] = focal_length[:, 1] * 300
focal_length = focal_length.cuda()

principal_point = torch.ones(8, 2)
principal_point[:, 0] = principal_point[:, 0] * 512
principal_point[:, 1] = principal_point[:, 1] * 256
principal_point = principal_point.cuda()

R = Rotation.from_euler("zyx", [[n * 5, n, n] for n in range(-4, 4, 1)], degrees=True).as_matrix()
R = torch.from_numpy(R).cuda()
T = [[n, 0, 0] for n in range(-4, 4, 1)]
T = torch.FloatTensor(T).cuda()

# print('R = ', R)
# exit()

camera = PerspectiveCameras(
    focal_length=focal_length,
    principal_point=principal_point,
    in_ndc=False,
    image_size=image_size,
    R=R,
    T=T,
    device="cuda",
)

world_to_view_transform = camera.get_world_to_view_transform()
world_to_screen_transform = camera.get_full_projection_transform()
print("The type of world_to_screen_transform = ", type(world_to_screen_transform))

# Load meshes using PyTorch3D
vertices, faces, aux = load_obj(mesh_file)
vertices = vertices.cuda()

world_to_view_vertices = world_to_view_transform.transform_points(vertices)
world_to_screen_vertices = world_to_screen_transform.transform_points(vertices)


# print("world_to_view_vertices = ", world_to_view_vertices)
# print("world_to_screen_vertices = ", world_to_screen_vertices)

visualizing the mesh using open3D
The type of world_to_screen_transform =  <class 'pytorch3d.transforms.transform3d.Transform3d'>
